+++
title = "Working with (Vanilla) PyTorch"
+++

This guide goes through how to use this package with the standard PyTorch workflow.

In [ ]:
!pip3 install tabben torch

## Loading the data

For this example, we'll use the poker hand prediction dataset.

In [2]:
from tabben.datasets import OpenTabularDataset

ds = OpenTabularDataset('./data/', 'poker')

  0%|          | 0/6867096 [00:00<?, ?it/s]

And let's just look at the input and output attributes:

In [2]:
print('Input Attribute Names:')
print(ds.input_attributes)

print('Output Attribute Names:')
print(ds.output_attributes)

Input Attribute Names:
['S1' 'C1' 'S2' 'C2' 'S3' 'C3' 'S4' 'C4' 'S5' 'C5']
Output Attribute Names:
['label']


Since we're working with PyTorch, the `OpenTabularDataset` object above is a PyTorch `Dataset` object that we can directly feed into a `DataLoader`.

In [3]:
from torch.utils.data import DataLoader

dl = DataLoader(ds, batch_size=8)

example_batch = next(iter(dl))
print(f'Input Shape (Batched): {example_batch[0].shape}')
print(f'Output Shape (Batched): {example_batch[1].shape}')

Input Shape (Batched): torch.Size([8, 10])
Output Shape (Batched): torch.Size([8])


## Setting up a basic model

First, we'll create a basic model in PyTorch, just for illustration (you can replace this with whatever model you're trying to train/evaluate). It'll just be a feedforward neural network with a couple dense/linear layers (this probably won't perform well).

In [4]:
from torch import nn
import torch.nn.functional as F


class ShallowClassificationNetwork(nn.Module):
    
    def __init__(self, num_inputs, num_classes):
        super().__init__()
        self.linear1 = nn.Linear(num_inputs, 32)
        self.linear2 = nn.Linear(32, 32)
        self.linear3 = nn.Linear(32, num_classes)
    
    def forward(self, inputs):
        # [b, num_inputs] -> [b, 32]
        x = F.relu(self.linear1(inputs))
        
        # [b, 32] -> [b, 32]
        x = F.relu(self.linear2(x))
        
        # [b, 32] -> [b, num_classes] (log(softmax(.)) computed over each row)
        x = F.log_softmax(self.linear3(x), dim=1)
        
        return x
    
    def predict(self, inputs):
        x = F.relu(self.linear1(inputs))
        x = F.relu(self.linear2(x))
        return F.softmax(self.linear3(x), dim=1)


In [5]:
import torch

device = torch.device('cpu')  # change this to 'cuda' if you have access to a CUDA GPU
model = ShallowClassificationNetwork(ds.num_inputs, ds.num_classes).to(device)

## Training the model

Now that we have a basic model and a training dataset (the default split for `OpenTabularDataset` is the train split), we can train our simple network using a PyTorch training loop.

In [6]:
from torch import optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)


In [7]:
from tqdm import trange

model.train()

training_progress = trange(30, desc='Train epoch')
for epoch in training_progress:
    running_loss = 0.
    running_acc = 0.
    running_count = 0
    
    for batch_input, batch_output in dl:
        optimizer.zero_grad()
        
        outputs = model(batch_input.float().to(device))
        preds = outputs.argmax(dim=1)
        
        loss = criterion(outputs, batch_output)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        running_acc += (preds == batch_output).sum()
        running_count += batch_input.size(0)
    
    training_progress.set_postfix({
        'train loss': f'{running_loss / running_count:.4f}',
        'train acc': f'{100 * running_acc / running_count:.2f}%',
    })


Train epoch: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 553.88it/s, train loss=0.0089, train acc=100.00%]


You can play around with the hyperparameters, but the model isn't likely to get particularly good performance like this. But, we'll go ahead and evaluate the final model (we ignored having a validation set in this guide) on the test set.

## Evaluating on the test set

In [19]:
test_ds = OpenTabularDataset('./data/', 'poker', split='test')
test_dl = DataLoader(test_ds, batch_size=16)
print(len(test_ds))

Data already available at `data/poker.npz`
10


Let's run the model and save its outputs for later evaluation.

In [13]:
model.eval()

pred_outputs = []
gt_outputs = []

for test_inputs, test_outputs in test_dl:
    batch_outputs = model(test_inputs.float().to(device))
    pred_outputs.append(batch_outputs.detach().cpu())
    gt_outputs.append(test_outputs)

In [14]:
test_pred_outputs = torch.vstack(pred_outputs).detach().cpu()
test_gt_outputs = torch.hstack(gt_outputs).detach().cpu()
print(test_pred_outputs)
print(test_gt_outputs)

tensor([[-1.5647e+01, -1.6075e+01, -1.5923e+01, -1.5735e+01, -1.5817e+01,
         -1.5155e+01, -1.5495e+01, -1.4744e+01, -1.6896e-03, -6.3850e+00],
        [-1.0643e+01, -1.0917e+01, -1.0864e+01, -1.0734e+01, -1.0827e+01,
         -1.0255e+01, -1.0533e+01, -1.0003e+01, -7.0259e-03, -4.9921e+00],
        [-1.0501e+01, -1.0780e+01, -1.0698e+01, -1.0526e+01, -1.0748e+01,
         -1.0116e+01, -1.0391e+01, -9.8666e+00, -6.9794e-03, -5.0038e+00],
        [-1.3968e+01, -1.4266e+01, -1.4205e+01, -1.4096e+01, -1.4091e+01,
         -1.3400e+01, -1.3740e+01, -1.3106e+01, -2.6051e-03, -5.9548e+00],
        [-9.1742e+00, -9.8402e+00, -9.8968e+00, -8.9129e+00, -8.7744e+00,
         -8.9716e+00, -8.6310e+00, -9.2210e+00, -3.6032e+00, -2.8542e-02],
        [-1.5139e+01, -1.6639e+01, -1.6175e+01, -1.4354e+01, -1.4642e+01,
         -1.5504e+01, -1.3987e+01, -1.5875e+01, -1.1814e+01, -1.0014e-05],
        [-2.1373e+01, -2.3500e+01, -2.2741e+01, -2.0185e+01, -2.0689e+01,
         -2.1936e+01, -1.9725e+0

In [15]:
from sklearn.metrics import roc_auc_score
print(ds.num_classes)
roc_auc_score(test_pred_outputs, test_gt_outputs.int())

10


ValueError: continuous-multioutput format is not supported

We can get the standard set of metrics and then evaluate the outputs of the test set on them.

In [ ]:
from tabben.evaluators import get_metrics

eval_metrics = get_metrics(ds.task, classes=ds.num_classes)
for metric in eval_metrics:
    print(metric)
    print(f'{metric}: {metric(test_pred_outputs, test_gt_outputs)}')

---

This code was last run with the following versions (if you're looking at the no-output webpage, see the notebook in the repository for versions):

In [18]:
from importlib.metadata import version

packages = ['torch', 'tabben']

for pkg in packages:
    print(f'{pkg}: {version(pkg)}')

torch: 1.10.0
tabben: 0.0.3.dev0
scikit-learn: 1.0.1
